# 1週間分のルールを作成するためのprogram

In [18]:
#加工前データ
import xlrd
import pprint
import openpyxl
import numpy as np
from datetime import datetime as dt
import pandas as pd
from sklearn import preprocessing
import copy

In [19]:
bed_data_filename = 'bed_data.csv'
before_csv = 'osaka_data_before.csv'
data_filename = 'osaka_data_before.xlsx'
save_filename = 'before_1week_MA.xlsx'
data_sheet = 'Sheet1'

In [20]:
#開始日と終了日を参照する
df = pd.read_csv('./data/'+bed_data_filename, encoding='shift-jis', index_col=0, parse_dates=['日付'])
start_day = df.index[0]
end_day = df.index[-1]
start_day, end_day

(Timestamp('2020-07-30 00:00:00'), Timestamp('2021-09-18 00:00:00'))

In [21]:
test_start = dt(2020,10,1,0)
#test_end = dt(2021,1,11,0)
day_num = (end_day-start_day).days

#diff
total_data_num = day_num+1 -1 
test_data_num = (end_day-test_start).days+1 -1

teach_data_num = total_data_num-test_data_num

#メンバシップ関数の数
func_num = 3
#入力変数
x_dimension = 6
#学習する変数の数
total_dimension = func_num**x_dimension
#学習回数
max_iter = 100
omega_iter = 100
#学習係数
l_rate = 0.5

In [22]:
#データ関連関数
def get_value_list(t_2d):
    return([[cell.value for cell in row] for row in t_2d])

def get_list_2d(sheet, start_row, end_row, start_col, end_col):
    return get_value_list(sheet.iter_rows(min_row=start_row,
                                          max_row=end_row,
                                          min_col=start_col,
                                         max_col=end_col))

def write_list_2d(sheet, l_2d, start_row, start_col):
    for y, row in enumerate(l_2d):
        for x, cell in enumerate(row):
            sheet.cell(row=start_row + y,
                       column=start_col + x,
                       value=l_2d[y][x])

In [23]:
#μやωからyを計算
def cal_y(m, total_m, y, w, omega):
    temp = 0
    for (i, j, l) in zip(m, w, omega):
        temp += i * j * l
    y[1] = temp/total_m

In [24]:
#データを扱いやすく変形
def set_data(y, Y, data, b, c, omega): 
    for i in range(total_data_num):
        y.append([(Y[i][0]-np.min(Y))/(np.max(Y)-np.min(Y)),(np.mean(Y)-np.min(Y))/(np.max(Y)-np.min(Y))])
        
    for i in range(total_dimension):
        omega[i] = (i+1)/total_dimension

In [25]:
#メンバーシップ関数
def fun(c, b, fun_data, data):    
    for i in range(total_data_num):
        for j in range(x_dimension):
            for k in range(func_num):
                #print(i,j,data[i][j])
                if data[i][j] < c[k]-b[k]:
                    fun_data[i][k][j] = 0
                elif data[i][j] > c[k]+b[k]:
                    fun_data[i][k][j] = 0
                elif data[i][j] >= c[k]-b[k] and data[i][j] <= c[k]:
                    fun_data[i][k][j] = 1-abs(c[k]-data[i][j])*(1/b[k])
                elif data[i][j] > c[k] and data[i][j] <= c[k]+b[k]:
                    fun_data[i][k][j] = 1-abs(c[k]-data[i][j])*(1/b[k])

In [26]:
#データに関数を適用する関数
def rule(m, total_m, fun_data, data, c, b, w):
    fun(c, b, fun_data, data)
    
    for d_num in range(total_data_num):
        for j in range(x_dimension):
            times = 0
            R = func_num**(x_dimension-1-j)
            for p in range(R):
                for i in range(func_num):
                    r = func_num**j
                    for k in range(r):
                        m[d_num][times] *= fun_data[d_num][i][j]
                        times+=1
                        
    #学習用とテスト用のμの総和
    for j in range(total_data_num):
        for i in range(total_dimension):
            total_m[j] += m[j][i] * w[i]

In [27]:
#最急降下法 改良
def gradient_descent(total_m, m, y, w, omega, mse):
    
    #累計学習回数 ※データ数ではなく学習した週の数
    for week in range(int(total_data_num/7)):
        #学習によって書き換えが生じてしまため、コピーを作成
        temp_y = copy.deepcopy(y)
        temp_tm = copy.deepcopy(total_m)
        temp_m = copy.deepcopy(m)
        temp_w = copy.deepcopy(w)
        temp_omega = copy.deepcopy(omega)
        
        #後件部の学習
        for i in range(omega_iter):
            for j in range(7):
                for dim in range(total_dimension):
                    temp_omega[dim] -= l_rate * (-temp_y[week*7+j][0]+temp_y[week*7+j][1]) * temp_m[week*7+j][dim] / temp_tm[week*7+j]
                    temp_w[dim] = temp_w[dim] - l_rate *  (-temp_y[week*7+j][0]+temp_y[week*7+j][1]) * temp_m[week*7+j][dim] * (temp_omega[dim]-temp_y[week*7+j][1]) / temp_tm[week*7+j]
                temp1 = 0
                temp2 = 0
                for r, W, ome in zip(temp_m[week*7+j], temp_w, temp_omega):
                    temp1 += r * W * ome
                    temp2 += r * W
                temp_tm[week*7+j] = temp2
                temp_y[week*7+j][1] = temp1/temp_tm[week*7+j]
        save_result(temp_y, temp_w, temp_omega, temp_m, temp_tm, week, mse)
    print("後件部学習終了")
    print("重み学習終了")
    
    '''#testデータで
    for i in range(teach_data_num,total_data_num):
        temp = 0
        for l,r in zip(m[i],w):
            temp += l*r
            if r > 0.75:
                temp += l*r
        total_m[i] = temp
        
        #エラー処理
        if total_m[i] == 0:
            print(CV_num,i)

        temp = 0
        for r, W, ome in zip(m[i], w, omega):
            temp += r * W * ome
        y[i][1] = temp/total_m[i]'''

    #MES(y, mse, max_iter, w, omega, 'test')
    #save_result(y, w, omega, m, total_m, 2)

In [28]:
def MES(y, mse, week_num):
    
    temp = 0
    for i in range(week_num*7, (week_num+1)*7):
        temp += (y[i][1] - y[i][0])**2
    mse[week_num] = temp/7 #7は1週間分という意味
    
    return 

In [29]:
#excelの保存
def save_result(y, w, omega, m, total_m, week_num, mse):
    
    wc = openpyxl.load_workbook('./output/osaka/'+save_filename, data_only=True)
    
    y_sheet = wc['y']
    if week_num == 0:
        write_list_2d(y_sheet,[['理想値week'+str(week_num),'week'+str(week_num)]], 1, 1)
        write_list_2d(y_sheet,y[:,:1], 2, 1)
        write_list_2d(y_sheet,y[:7,1:], 2, 2)
    else:
        write_list_2d(y_sheet,[['week'+str(week_num)]], 1, 2+week_num)
        write_list_2d(y_sheet,y[week_num*7:(week_num+1)*7,1:], 2+week_num*7, 2+week_num)
    
    MES(y, mse, week_num)
    mse_sheet = wc['mse']
    
    write_list_2d(mse_sheet,[['week'+str(week_num)]], 1, 1+week_num)
    mse_sheet.cell(row=2, column=1+week_num).value = mse[week_num]
    
    wc.save('./output/osaka/'+save_filename)
    print("Save complited.week"+str(week_num))
    
    return 

In [30]:
#excelの保存
def save_fix_result(fix):
    wc = openpyxl.load_workbook('./output/osaka/'+save_filename, data_only=True)
    for sheet_name in wc.sheetnames:
        if sheet_name == 'output':
            fix_sheet = wc["fix"]
            write_list_2d(fix_sheet,[['理想値','出力値']], 1, 1)
            write_list_2d(fix_sheet,fix, 2, 1)
    wc.save('./output/osaka/'+save_filename)
    print("Already exist. Save complited ")  
    return 

In [31]:
#出力yを導出する関数
def make_output(total_m, m, y, fun_data, data, omega, c, b, w, mse):
    
    print("学習開始")
    gradient_descent(total_m, m, y, w, omega, mse)
    print("学習完了")
    
    return 0

In [32]:
# csvファイルを読み込む
df_csv = pd.read_csv('./data/'+before_csv, encoding='shift-jis')

csv_data = df_csv.drop(['日付'], axis=1)

mm = preprocessing.MinMaxScaler()
mm_data = mm.fit_transform(csv_data)

In [33]:
csv_data

,検査件数,陽性人数,現在陽性者数,退院,死亡,リンク不明者,病床使用率
0,2070,190,1154,121,0,135,21.0
1,2328,216,1265,104,1,146,23.0
2,2293,195,1342,118,0,137,24.0
3,1562,194,1445,91,0,142,29.0
4,1266,81,1423,102,1,55,30.0
...,...,...,...,...,...,...,...
411,6947,942,15603,2285,4,556,60.0
412,16708,1160,14684,2075,9,685,56.0
413,16887,858,13765,1775,12,518,54.0
414,20092,735,12749,1755,5,445,52.0


In [34]:
#自身を中心に前後３日間の１週間分のデータで平均化
test = csv_data.rolling(7, center=True, min_periods=4).mean().round(1)
Y = test['病床使用率'].values
Y = Y.reshape(-1,1)
test.drop('病床使用率', axis=1, inplace=True)
test

,検査件数,陽性人数,現在陽性者数,退院,死亡,リンク不明者
0,2063.2,198.8,1301.5,108.5,0.2,140.0
1,1903.8,175.2,1325.8,107.2,0.4,123.0
2,1983.0,178.2,1356.3,107.0,0.5,124.0
3,2000.7,180.7,1382.3,115.6,0.4,123.7
4,2059.4,185.7,1432.3,135.1,0.6,125.4
...,...,...,...,...,...,...
411,15621.9,936.7,15953.0,2038.4,5.1,556.9
412,15227.9,851.4,14825.0,1978.7,6.3,479.9
413,15039.2,802.2,14213.0,1962.8,7.2,447.0
414,15508.2,872.2,13667.8,1952.0,7.8,489.4


In [35]:
test.replace([np.inf, -np.inf], np.nan, inplace=True)
test.dropna(axis=0, how='any', inplace=True)
test

,検査件数,陽性人数,現在陽性者数,退院,死亡,リンク不明者
0,2063.2,198.8,1301.5,108.5,0.2,140.0
1,1903.8,175.2,1325.8,107.2,0.4,123.0
2,1983.0,178.2,1356.3,107.0,0.5,124.0
3,2000.7,180.7,1382.3,115.6,0.4,123.7
4,2059.4,185.7,1432.3,135.1,0.6,125.4
...,...,...,...,...,...,...
411,15621.9,936.7,15953.0,2038.4,5.1,556.9
412,15227.9,851.4,14825.0,1978.7,6.3,479.9
413,15039.2,802.2,14213.0,1962.8,7.2,447.0
414,15508.2,872.2,13667.8,1952.0,7.8,489.4


In [36]:
test.max()

検査件数      20066.3
陽性人数       2518.1
現在陽性者数    27085.4
退院         2680.1
死亡           37.6
リンク不明者     1725.0
dtype: float64

In [37]:
test.min()

検査件数      1256.4
陽性人数        47.1
現在陽性者数     457.1
退院          46.3
死亡           0.2
リンク不明者      29.1
dtype: float64

In [38]:
index_num = [n for n in test.index]
#削除されたindex　numberの確認
#病床からも削除をする
drop_num = []
for i in range(len(csv_data)):
    if not i in index_num:
        drop_num.append(i)
drop_num

[]

In [39]:
test = mm.fit_transform(test)
print(test.max(), test.min())

1.0 0.0


In [40]:
test_y = mm.fit_transform(Y)
test_y = np.delete(test_y, drop_num, axis=0)

In [41]:
#中心と幅の初期値の設定
c = np.array([0, 0.5, 1])
b = np.array([1, 0.5, 1]) #変えるべき

total_data_num = day_num+1 - len(drop_num)

#データの準備
fun_data = np.empty((total_data_num, func_num, x_dimension))
m = np.ones((total_data_num, total_dimension))
total_m = np.zeros(total_data_num)
omega = [(i+1)/total_dimension for i in range(total_dimension)]
w = np.ones(total_dimension)
mse = np.zeros(int(total_data_num/7)) #各週の誤差を算出

y_mean = np.mean(test_y)
y = np.hstack((test_y[0:,:], np.full((len(test),1), y_mean)))
mm_data = test

rule(m, total_m, fun_data, mm_data, c, b, w)
print("データセット完了")

make_output(total_m, m, y, fun_data, mm_data, omega, c, b, w, mse)

データセット完了
学習開始
Save complited.week0
Save complited.week1
Save complited.week2
Save complited.week3
Save complited.week4
Save complited.week5
Save complited.week6
Save complited.week7
Save complited.week8
Save complited.week9
Save complited.week10
Save complited.week11
Save complited.week12
Save complited.week13
Save complited.week14
Save complited.week15
Save complited.week16
Save complited.week17
Save complited.week18
Save complited.week19
Save complited.week20
Save complited.week21
Save complited.week22
Save complited.week23
Save complited.week24
Save complited.week25
Save complited.week26
Save complited.week27
Save complited.week28
Save complited.week29
Save complited.week30
Save complited.week31
Save complited.week32
Save complited.week33
Save complited.week34
Save complited.week35
Save complited.week36
Save complited.week37
Save complited.week38
Save complited.week39
Save complited.week40
Save complited.week41
Save complited.week42
Save complited.week43
Save complited.week44
Save c

0